In [27]:
# Importamos las librerías a usar
import pandas as pd
import numpy as np
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [28]:
def load_and_preprocess_data(file_path):
    """
    Carga un dataset de juegos y realiza un preprocesamiento para su uso en sistemas de recomendación.

    Esta función lleva a cabo el preprocesamiento necesario para dos sistemas de recomendación diferentes:
    ítem-ítem y usuario-ítem. Para ítem-ítem, combina géneros y desarrolladores de juegos y utiliza
    TF-IDF para transformarlos en vectores numéricos, luego calcula la similitud del coseno entre los juegos.
    Para usuario-ítem, agrupa los datos por usuario y género y normaliza el tiempo total de juego,
    calculando luego la similitud del coseno entre los patrones de juego de los usuarios.

    Args:
        file_path (str): Ruta al archivo CSV que contiene los datos del juego.

    Returns:
        tuple: Contiene tres elementos en el siguiente orden:
            - DataFrame pandas con los datos del juego cargados.
            - Matriz de similitud del coseno para el sistema de recomendación ítem-ítem.
            - Matriz de similitud del coseno para el sistema de recomendación usuario-ítem.
    """
    
    df = pd.read_csv(file_path)
    
    # Eliminar duplicados basados en 'item_id'
    df_item = df.drop_duplicates(subset='item_id').copy() 

    """
     Sistema de Recomendación Item - Item

    - El preprocesamiento del dataset se enfocará en el género y el desarrollador de cada uno de los registros

    - Esta es una elección del diseño del modelo basado en la relevancia de estas características para para determinar 
        la similitud entre los juegos.

    - *Género (Genre)*: Es una de las características más descriptivas y diferenciadoras de un juego. 
        Los usuarios a menudo tienen preferencias claras en cuanto a géneros, por lo que es un buen predictor de lo que podría gustarles.

    - *Desarrollador (Developer)*: Algunos jugadores son seguidores de ciertos desarrolladores y 
        tienden a disfrutar de otros juegos del mismo creador debido a un estilo, calidad o tema.

    - Al enfocarnos en un número limitado de características, el modelo puede ser más simple y eficiente, 
        mientras que proporciona recomendaciones útiles y precisas.

    - Ayuda también al problema de la "maldición de la dimensionalidad", donde demasiadas características pueden 
        hacer el análisis menos efectivo y más intensivo en computo.
    """
    
    # Preprocesamiento de datos para ítem-ítem
    
    # Combinamos los géneros y los desarrolladores de juegos en una sola cadena de texto
    df_item['combined_features'] = df['genres'] + " " + df['developer']
    
    # Utilizamos TF-IDF(Frecuencia de término - frecuencia inversa del documento) 
    # para convertir el texto en un conjunto de vectores numéricos
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_item['combined_features'])
    
    # Calculamos la similitud del coseno entre estos vectores para entender qué tan similares son los juegos entre sí
    cosine_sim_item = cosine_similarity(tfidf_matrix)

    """
    Sistema de Recomendación Usuario - Item

    - El preprocesamiento para este sistema se enfocará en el identificador del juego('item-id'), género('genre') y 
        tiempo de juego ('playtime_forever').

    - *Identificador de juego ('item-id')*: necesario para identificar cada juego individual en el dataset y relacionar 
        los juegos con los usuarios.

    - *Género ('genre')*: Uno de los factores más importantes que influyen en las preferencias de los usuarios. 
        Los usuarios a menudo tienen géneros favoritos y es probable que disfruten otros juegos dentro del mismo género. 

    - *Tiempo total de juego ('playtime_forever')*: esta medida proporciona cuánto tiempo un usuario ha dedicado a cada juego, lo cual puede ser un indicador fuerte de su preferencia, más que la reseña que es una acción opcional. Asume que los juegos en los que un usuario ha invertido más tiempo probablemente sean aquellos que más le gustan.

    - *Disponibilidad de Datos*: estas características pueden ser las más consistentemente disponibles y confiables en el conjunto de datos.

    - Estas tres características juntas permiten crear un perfil de preferencias para cada usuario basado en los tipos de juegos que juegan y cuánto tiempo pasan jugándolos. La idea es que si dos usuarios han dedicado cantidades de tiempo similares a géneros similares de juegos, es probable que tengan preferencias similares y, por lo tanto, podrían disfrutar de los mismos juegos.
    """
    # Preprocesamiento de datos para usuario-Ítem
    
    """
    Debido a las limitaciones de memoria de la API, implementaremos un filtrado para los usuarios que más juegos tienen y
    llevan más tiempo total de horas jugadas, esto nos resultará en una muestra de los datos del dataframe original 
    para resolver el sistema de recomendación en un servidor virtual
    """
    
    # Calcular métricas clave
    user_metrics = df.groupby('user_id').agg(
        tiempo_total_jugado=pd.NamedAgg(column='playtime_forever', aggfunc='sum'),
        items_count=pd.NamedAgg(column='items_count', aggfunc='max')
    ).reset_index()

    # Ordenar usuarios por número de juegos jugados (items_count) y seleccionar los primeros 'max_usuarios'
    usuarios_seleccionados = user_metrics.sort_values(by='items_count', ascending=False).head(7000)

    # Obtener los IDs de los usuarios seleccionados
    usuarios_seleccionados_ids = usuarios_seleccionados['user_id']

    # Filtrar el DataFrame original para incluir solo los usuarios seleccionados
    df = df[df['user_id'].isin(usuarios_seleccionados_ids)]
    
    # Agrupamos los datos por usuario y género y sumamos el tiempo total del juego por género
    user_genre_playtime = df.groupby(['user_id', 'genres'])['playtime_forever'].sum().unstack(fill_value=0)
    
    # Normalizamos estos datos para que cada fila sume 1, lo que nos da la proporción del tiempo dedicado a cada género
    # por usuario
    user_genre_playtime_normalized = user_genre_playtime.div(user_genre_playtime.sum(axis=1), axis=0)
    user_genre_playtime_normalized = user_genre_playtime_normalized.fillna(0).replace([np.inf, -np.inf], 0)
    
    # Calculamos la similitud del coseno entre usuarios para entender qué tan similares son en términos de preferencia de juego
    user_similarity_user = cosine_similarity(user_genre_playtime_normalized)
    
    # Convirtiendo la matriz de similitud a un DataFrame
    user_similarity_df = pd.DataFrame(user_similarity_user, index=user_genre_playtime_normalized.index, columns=user_genre_playtime_normalized.index)


    return df, df_item, cosine_sim_item, user_similarity_df

In [29]:
# Función para recomendación ítem-ítem
def recomendacion_juego(item_id, df, cosine_sim):
    """
    Genera una lista de juegos recomendados similares a un juego específico.

    Esta función identifica juegos similares a partir de un juego dado, utilizando la similitud del coseno 
    basada en características combinadas como géneros y desarrolladores. La función devuelve los cinco 
    juegos más similares, excluyendo el juego de entrada.

    Args:
        item_id (int): El ID del juego para el cual se harán las recomendaciones.
        df (pd.DataFrame): El DataFrame que contiene los datos de los juegos, incluyendo 'item_id', 'genres', y 'developer'.
        cosine_sim (numpy.ndarray): Matriz de similitud del coseno precalculada para los juegos.

    Returns:
        list of dict: Una lista de diccionarios, donde cada diccionario contiene 'item_id' y 'app_name' 
                      de los juegos recomendados. Devuelve una lista vacía si el juego no se encuentra en el dataset.
    
    Ejemplo:
        recomendaciones = recomendacion_juego(123, df, cosine_sim)
        # Esto podría devolver juegos similares al juego con ID 123.
    """
    
    # Si el juego no está en el DataFrame, devuelve un mensaje de error.
    if item_id not in df['item_id'].values:
        return "El juego con el ID proporcionado no se encuentra en el dataset."
    
    # Busca el índice del juego en el DataFrame usando el ID proporcionado.
    idx = df.index[df['item_id'] == item_id].tolist()[0]
    
    # Crea una lista de pares (índice, puntuación de similitud) para todos los juegos, 
    # basándose en la fila correspondiente al juego en la matriz de similitud.
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordena los juegos de acuerdo a su puntuación de similitud, de mayor a menor.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Selecciona los primeros 5 juegos (excluyendo el propio juego que es el más similar a sí mismo).
    sim_scores = sim_scores[1:6]  # El primero es el propio juego
    
    # Extrae los índices de los juegos recomendados de los pares (índice, puntuación).
    game_indices = [i[0] for i in sim_scores]
    
    # Utiliza los índices para obtener los IDs y nombres de los juegos recomendados del DataFrame y los devuelve.
    recomendaciones = df.iloc[game_indices][['item_id', 'app_name']].to_dict('records')
    return recomendaciones


In [30]:
# Recibe el ID del usuario, el DataFrame, la matriz de similitud entre usuarios y el número de recomendaciones deseado.
def recomendacion_usuario(user_id, df, user_similarity_df, num_recommendations=5):
    """
    Genera recomendaciones de juegos para un usuario específico basándose en usuarios similares.

    Esta función busca usuarios con patrones de juego similares al usuario dado y recomienda juegos que 
    estos usuarios similares han jugado, pero que el usuario en cuestión aún no ha probado. Utiliza la 
    matriz de similitud del coseno entre usuarios para determinar qué usuarios son similares.

    Args:
        user_id (int): El ID del usuario para el cual se realizará la recomendación.
        df (pd.DataFrame): El DataFrame que contiene los datos de los juegos y usuarios.
        user_similarity_df (pd.DataFrame): DataFrame que representa la matriz de similitud del coseno entre usuarios.
        num_recommendations (int, opcional): Número de recomendaciones a generar. Por defecto es 5.

    Returns:
        list of dict: Una lista de diccionarios, donde cada diccionario contiene 'item_id' y 'app_name' 
                      de los juegos recomendados. Devuelve una lista vacía si el usuario no se encuentra en el dataset.

    Ejemplo:
        recomendaciones_usuario = recomendacion_usuario_mod(456, df, user_similarity_df)
        # Esto podría devolver juegos recomendados para el usuario con ID 456.
    """
    
    # Si el usuario no está en el DataFrame, devuelve un mensaje de error.
    if user_id not in user_similarity_df.index:
        return "El usuario con el ID proporcionado no se encuentra en el dataset."

    # Ordena a los usuarios en función de su similitud con el usuario objetivo y selecciona los más similares
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]
    
    # Crea un conjunto de IDs de juegos que el usuario objetivo ya ha jugado.
    user_games = set(df[df['user_id'] == user_id]['item_id'])
    
    # Recorre los usuarios similares, recopilando juegos que ellos han jugado pero el usuario objetivo no. 
    # Detiene el bucle una vez que se alcanza el número deseado de recomendaciones.
    recommended_games = set()
    for similar_user in similar_users:
        similar_user_games = set(df[df['user_id'] == similar_user]['item_id'])
        new_recommendations = similar_user_games.difference(user_games)
        
        # Actualiza el conjunto de juegos recomendados, pero solo hasta alcanzar num_recommendations
        recommended_games.update(new_recommendations)
        if len(recommended_games) >= num_recommendations:
            break
        
    # Se asegura que sólo devuelva exactamente el número de juegos recomendados
    final_recommendations = list(recommended_games)[:num_recommendations]
    recomendaciones = df[df['item_id'].isin(final_recommendations)].drop_duplicates(subset='item_id')[['item_id', 'app_name']].to_dict('records')

    return recomendaciones

In [31]:
# Cargar y preprocesar los datos
file_path = '../src/data/dataset_full.csv'
df, df_item, cosine_sim_item, user_similarity_df = load_and_preprocess_data(file_path)

In [36]:
recomendaciones = recomendacion_juego(377160, df_item, cosine_sim_item)
recomendaciones

[{'item_id': 323850, 'app_name': 'Move or Die'},
 {'item_id': 242800, 'app_name': "Day One : Garry's Incident"},
 {'item_id': 247730, 'app_name': 'Nether: Resurrected'},
 {'item_id': 330580, 'app_name': 'Shark Attack Deathmatch 2'},
 {'item_id': 241720, 'app_name': 'Guncraft'}]

In [37]:
recomendaciones = recomendacion_usuario('diego9031', df, user_similarity_df)
recomendaciones

[{'item_id': 319510, 'app_name': "Five Nights at Freddy's"},
 {'item_id': 262060, 'app_name': 'Darkest Dungeon®'},
 {'item_id': 248860, 'app_name': 'NEO Scavenger'},
 {'item_id': 247000, 'app_name': 'Talisman: Digital Edition'},
 {'item_id': 227860, 'app_name': 'Castle Story'}]

In [34]:
# Ejemplo para un DataFrame de pandas
df_memory = df.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"El uso de memoria de df es aproximadamente {df_memory:.2f} MB")

# Ejemplo para una matriz de NumPy o matriz dispersa
# Supongamos que tfidf_matrix es una matriz dispersa y user_similarity_user es un array de NumPy
cosine_sim_item_memory = sys.getsizeof(cosine_sim_item) / (1024 ** 2)
user_similarity_memory = sys.getsizeof(user_similarity_df) / (1024 ** 2)

print(f"El uso de memoria de cosine_sim_item es aproximadamente {cosine_sim_item_memory:.2f} MB")
print(f"El uso de memoria de user_similarity_df es aproximadamente {user_similarity_memory:.2f} MB")


El uso de memoria de df es aproximadamente 2.29 MB
El uso de memoria de cosine_sim_item es aproximadamente 19.41 MB
El uso de memoria de user_similarity_df es aproximadamente 374.56 MB
